In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycaret
from pycaret.classification import *
from pycaret.datasets import get_data

In [13]:
%pip install 'pycaret'

Note: you may need to restart the kernel to use updated packages.


In [14]:

%pip install 'pycaret[full]'

  Using cached Flask-2.2.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached werkzeug-2.3.8-py3-none-any.whl.metadata (4.1 kB)
  Using cached boto3-1.34.55-py3-none-any.whl.metadata (6.6 kB)
  Using cached evidently-0.2.8-py3-none-any.whl.metadata (1.3 kB)
  Using cached explainerdashboard-0.4.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached fairlearn-0.7.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached fastapi-0.110.0-py3-none-any.whl.metadata (25 kB)
  Using cached fugue-0.8.7-py3-none-any.whl.metadata (17 kB)
  Using cached gradio-4.19.2-py3-none-any.whl.metadata (15 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached interpret-0.5.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached kmodes-0.12.2-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached m2cgen-0.10.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached mlflow-1.30.1-py3-none-any.whl.metadata (11 kB)
  Using cached mlxtend-0.23.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached mot

In [93]:
data = pd.read_csv('train.csv')
data

,DATETIMEDATA,PM25,O3,WS,TEMP,RH,WD
0,2024-01-01 00:00:00,16.5,16.000000,1,27.0,79.0,115.0
1,2024-01-01 01:00:00,33.6,11.326513,1,27.0,82.0,97.0
2,2024-01-01 02:00:00,34.2,11.326513,1,26.0,84.0,96.0
3,2024-01-01 03:00:00,20.1,11.326513,1,26.0,82.0,108.0
4,2024-01-01 04:00:00,16.5,1.000000,1,27.0,79.0,106.0
...,...,...,...,...,...,...,...
1411,2024-02-28 19:00:00,23.6,11.326513,1,29.0,64.0,184.0
1412,2024-02-28 20:00:00,25.2,11.326513,1,29.0,64.0,183.0
1413,2024-02-28 21:00:00,23.2,1.000000,1,29.0,64.0,159.0
1414,2024-02-28 22:00:00,24.7,3.000000,1,28.0,65.0,134.0


In [94]:

from pycaret.regression import *
clf1 = setup(data, target='PM25', session_id = 123)

,Description,Value
0,Session id,123
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1416, 7)"
4,Transformed data shape,"(1416, 7)"
5,Transformed train set shape,"(991, 7)"
6,Transformed test set shape,"(425, 7)"
7,Numeric features,5
8,Categorical features,1
9,Preprocess,True


In [95]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
en,Elastic Net,4.1559,28.3896,5.3160,0.0149,0.2866,0.2629,0.0080
huber,Huber Regressor,4.1462,28.6228,5.3380,0.0052,0.2878,0.2616,0.0120
omp,Orthogonal Matching Pursuit,4.1890,28.6899,5.3440,0.0044,0.2883,0.2654,0.0080
lasso,Lasso Regression,4.1705,28.7630,5.3514,0.0015,0.2883,0.2639,0.0070
llar,Lasso Least Angle Regression,4.1705,28.7630,5.3514,0.0015,0.2883,0.2639,0.0070
ridge,Ridge Regression,4.1769,28.9287,5.3670,-0.0044,0.2890,0.2642,0.0060
et,Extra Trees Regressor,4.1771,28.9355,5.3676,-0.0046,0.2890,0.2642,0.0270
gbr,Gradient Boosting Regressor,4.1789,28.9363,5.3677,-0.0047,0.2891,0.2644,0.0190
lar,Least Angle Regression,4.1776,28.9397,5.3680,-0.0048,0.2891,0.2642,0.0070
lr,Linear Regression,4.1776,28.9397,5.3680,-0.0048,0.2891,0.2642,0.0260


In [96]:
t_en = create_model('en')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.3714,28.8857,5.3745,0.0032,0.3175,0.3115
1,3.9175,30.0022,5.4774,-0.0096,0.2711,0.2325
2,4.1430,26.6804,5.1653,0.0016,0.2799,0.2580
3,4.3085,29.0552,5.3903,0.0257,0.2831,0.2594
4,4.1398,27.9136,5.2833,0.0238,0.2985,0.2755
5,4.3656,31.6537,5.6262,0.0233,0.2927,0.2666
6,3.9306,24.2149,4.9209,0.0109,0.2706,0.2448
7,4.1158,32.0332,5.6598,0.0195,0.2855,0.2470
8,3.5920,20.2372,4.4986,0.0427,0.2451,0.2186


In [97]:
tune_rf = tune_model(t_en)
bagged = ensemble_model(tune_rf , n_estimators=20)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.3351,28.5576,5.3439,0.0145,0.3153,0.3081
1,3.9189,30.0097,5.4781,-0.0099,0.2711,0.2323
2,4.1271,26.3426,5.1325,0.0142,0.2785,0.2571
3,4.2848,28.6154,5.3493,0.0404,0.2810,0.2578
4,4.1058,27.4461,5.2389,0.0402,0.2966,0.2739
5,4.3321,31.3012,5.5947,0.0342,0.2911,0.2647
6,3.9347,24.1967,4.9190,0.0117,0.2706,0.2450
7,4.0893,31.6891,5.6293,0.0301,0.2841,0.2457
8,3.5672,19.7230,4.4411,0.0670,0.2421,0.2169


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.3344,28.5631,5.3444,0.0143,0.3152,0.3078
1,3.9170,29.9757,5.4750,-0.0087,0.2708,0.2319
2,4.1257,26.3422,5.1325,0.0143,0.2784,0.2569
3,4.2833,28.6208,5.3498,0.0402,0.2809,0.2575
4,4.1050,27.4318,5.2375,0.0407,0.2963,0.2735
5,4.3282,31.2677,5.5917,0.0352,0.2908,0.2643
6,3.9360,24.2064,4.9200,0.0113,0.2707,0.2454
7,4.0952,31.7189,5.6320,0.0292,0.2844,0.2462
8,3.5670,19.7160,4.4403,0.0674,0.2420,0.2169


In [98]:
test = pd.read_csv('test.csv')
predict_model(bagged,data=test)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bagging Regressor,3.0506,14.5269,3.8114,-0.1770,0.1831,0.1497


,DATETIMEDATA,O3,WS,TEMP,RH,WD,PM25,prediction_label
0,2024-02-29 00:00:00,3.0,1,28.0,66.0,91.0,18.500000,18.628939
1,2024-02-29 01:00:00,24.0,1,27.0,68.0,96.0,16.100000,17.953123
2,2024-02-29 02:00:00,24.0,1,27.0,68.0,108.0,16.000000,17.884474
3,2024-02-29 03:00:00,26.0,1,27.0,68.0,98.0,15.400000,17.864035
4,2024-02-29 04:00:00,19.0,1,27.0,70.0,100.0,17.799999,18.082321
...,...,...,...,...,...,...,...,...
139,2024-03-05 19:00:00,5.0,1,29.0,69.0,116.0,23.299999,18.135102
140,2024-03-05 20:00:00,7.0,1,29.0,72.0,111.0,29.500000,18.023005
141,2024-03-05 21:00:00,1.0,1,29.0,73.0,108.0,28.500000,18.252090
142,2024-03-05 22:00:00,5.0,1,28.0,74.0,105.0,23.799999,18.303057


In [1]:
import pandas as pd
from pycaret.regression import *

# Read the training data
train_data = pd.read_csv('train.csv')


# Convert 'DATETIMEDATA' column to datetime format
train_data['DATETIMEDATA'] = pd.to_datetime(train_data['DATETIMEDATA'])

# Set up the PyCaret regression environment
regression_setup = setup(train_data, target='PM25', session_id=123, date_features=['DATETIMEDATA'], normalize=True)
best = compare_models()
# Create a Random Forest Regressor model
rf_model = create_model('rf')

# Fine-tune the Random Forest Regressor model
tuned_rf_model = tune_model(rf_model)

# Ensemble the tuned Random Forest Regressor model
bagged_model = ensemble_model(tuned_rf_model, n_estimators=20)

# Get the last prediction time
last_prediction_time = train_data['DATETIMEDATA'].max()

# Generate datetime values for every hour of each day from the last prediction time to the end of the next week
start_date = last_prediction_time + pd.DateOffset(hours=1)  # Start date for new predictions
end_date = start_date + pd.DateOffset(days=30, hours=23)  # End date for next week
next_week_hours = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a DataFrame with 'DATETIMEDATA' column for every hour of each day within the next week
next_week_data = pd.DataFrame({'DATETIMEDATA': next_week_hours})

# Add columns for 'O3', 'WS', 'TEMP', 'RH', 'WD' with placeholder values of 0
next_week_data['O3'] = 0
next_week_data['WS'] = 0
next_week_data['TEMP'] = 0
next_week_data['RH'] = 0
next_week_data['WD'] = 0

# Predict PM25 for every hour of each day within the next week
model_predictions = predict_model(bagged_model, data=next_week_data)

mpdf = pd.DataFrame(model_predictions, columns=['DATETIMEDATA', 'prediction_label'])

print(mpdf)

,Description,Value
0,Session id,123
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1416, 7)"
4,Transformed data shape,"(1416, 9)"
5,Transformed train set shape,"(991, 9)"
6,Transformed test set shape,"(425, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,3.2651,18.4389,4.2764,0.3586,0.2242,0.1976,0.0450
et,Extra Trees Regressor,3.2589,18.8994,4.3321,0.3408,0.2258,0.1954,0.0330
lightgbm,Light Gradient Boosting Machine,3.4173,20.2593,4.4864,0.2958,0.2353,0.2062,0.2400
gbr,Gradient Boosting Regressor,3.4846,20.3137,4.4987,0.2911,0.2385,0.2135,0.0170
knn,K Neighbors Regressor,3.5759,21.5973,4.6388,0.2429,0.2459,0.2163,0.0080
ada,AdaBoost Regressor,3.9284,23.9227,4.8734,0.1689,0.2661,0.2549,0.0110
lr,Linear Regression,3.9268,24.9870,4.9900,0.1302,0.2687,0.2455,0.1840
ridge,Ridge Regression,3.9270,24.9865,4.9899,0.1302,0.2687,0.2455,0.0060
lar,Least Angle Regression,3.9268,24.9870,4.9900,0.1302,0.2687,0.2455,0.0050
br,Bayesian Ridge,3.9314,25.0106,4.9921,0.1296,0.2690,0.2460,0.0050


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.7715,22.4174,4.7347,0.2264,0.2598,0.2410
1,3.3318,23.2404,4.8208,0.2179,0.2355,0.1978
2,3.3250,17.0587,4.1302,0.3617,0.2301,0.2061
3,3.5207,20.3910,4.5156,0.3162,0.2314,0.2029
4,2.8876,14.2023,3.7686,0.5033,0.2030,0.1772
5,3.3090,18.5945,4.3121,0.4263,0.2165,0.1899
6,2.8224,13.2914,3.6457,0.4571,0.1915,0.1628
7,3.2386,19.4643,4.4118,0.4042,0.2217,0.1902
8,2.8811,14.9033,3.8605,0.2950,0.1991,0.1716


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.9311,23.3562,4.8328,0.1940,0.2698,0.2562
1,3.3207,21.8756,4.6771,0.2639,0.2288,0.1949
2,3.4475,18.0110,4.2439,0.3260,0.2340,0.2120
3,3.7098,21.5134,4.6382,0.2786,0.2403,0.2156
4,3.1623,16.5569,4.0690,0.4210,0.2247,0.1988
5,3.3900,19.0599,4.3658,0.4119,0.2200,0.1961
6,2.9928,14.1363,3.7598,0.4226,0.2006,0.1769
7,3.5038,22.5833,4.7522,0.3088,0.2383,0.2066
8,2.8718,14.7430,3.8397,0.3026,0.1988,0.1703


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.7896,21.7701,4.6658,0.2487,0.2596,0.2458
1,3.3578,22.6790,4.7622,0.2368,0.2323,0.1979
2,3.3961,17.0513,4.1293,0.3619,0.2276,0.2090
3,3.5723,20.5190,4.5298,0.3119,0.2333,0.2076
4,3.0389,15.0337,3.8773,0.4742,0.2125,0.1894
5,3.3867,19.3917,4.4036,0.4017,0.2238,0.1992
6,2.9910,13.8561,3.7224,0.4340,0.1974,0.1749
7,3.4211,21.0806,4.5914,0.3548,0.2315,0.2030
8,2.7904,14.4427,3.8003,0.3168,0.1965,0.1665


   DATETIMEDATA  prediction_label
0    2024-02-29         16.904056
1    2024-03-01         21.028383
2    2024-03-02         20.995486
3    2024-03-03         20.137789
4    2024-03-04         20.217400
5    2024-03-05         21.244764
6    2024-03-06         21.144061
7    2024-03-07         21.407440
8    2024-03-08         21.681123
9    2024-03-09         21.903365
10   2024-03-10         21.786565
11   2024-03-11         21.733268
12   2024-03-12         21.827786
13   2024-03-13         21.496393
14   2024-03-14         21.010408
15   2024-03-15         20.225036
16   2024-03-16         20.084692
17   2024-03-17         19.982589
18   2024-03-18         19.928334
19   2024-03-19         19.975776
20   2024-03-20         21.862884
21   2024-03-21         22.242618
22   2024-03-22         18.135768
23   2024-03-23         15.046469
24   2024-03-24         15.092350
25   2024-03-25         15.403845
26   2024-03-26         16.498628
27   2024-03-27         16.656375
28   2024-03-2

In [100]:
mpdf.to_csv('model_predictions.csv', index=False)

In [1]:
evaluate_model(rf_model)

NameError: name 'evaluate_model' is not defined